In [ ]:
import os
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from ratelimit import limits, sleep_and_retry
from requests.exceptions import HTTPError, Timeout, RequestException
from urllib.parse import quote
from tqdm import tqdm
import asyncio
import aiohttp

# Load environment variables
load_dotenv()
API_KEY = os.getenv('api')

# Database connection
engine = create_engine('mssql+pyodbc://Revision-PC\\SQLEXPRESS/RiotDataDB?driver=ODBC+Driver+17+for+SQL+Server')

# Rate limit parameters
RATE_LIMIT = 100  # 100 requests per minute
TIME_WINDOW = 60  # 60 seconds

# Decorator to handle rate limiting
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_WINDOW)
def call_api(url):
    response = requests.get(url)
    response.raise_for_status()
    return response

# Function to retrieve summoner's PUUID
def get_puuid(summoner_name, summoner_tag):
    encoded_name = quote(summoner_name)
    api_url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encoded_name}/{summoner_tag}?api_key={API_KEY}"
    try:
        response = call_api(api_url)
        return response.json().get('puuid')
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")

# Function to retrieve match IDs
def get_match_ids(puuid):
    api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={API_KEY}"
    try:
        response = call_api(api_url)
        return response.json()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")

# Asynchronous function to retrieve match details
async def fetch_match_detail(session, match_id):
    api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={API_KEY}"
    try:
        async with session.get(api_url) as response:
            response.raise_for_status()
            return await response.json(), match_id
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return None, match_id

async def fetch_all_match_details(match_ids):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_match_detail(session, match_id) for match_id in match_ids]
        return await asyncio.gather(*tasks)

def process_match_details(matches_details):
    details_match = []
    for match_detail, match_id in tqdm(matches_details, desc="Processing match details"):
        if match_detail:
            queue_id = match_detail.get('info', {}).get('queueId')
            if queue_id == 420:  # Only consider ranked solo queue
                participants = match_detail.get('info', {}).get('participants', [])
                participants_df = pd.json_normalize(participants)
                participants_df.insert(0, 'Match_ID', match_id)
                details_match.append(participants_df)
    return pd.concat(details_match, ignore_index=True) if details_match else pd.DataFrame()

def main():
    # Get summoner details
    summoner_name = input("Escreva o nome do invocador: ")
    summoner_tag = input("Digite a tag do invocador: ")
    
    print("Retrieving PUUID...")
    puuid = get_puuid(summoner_name, summoner_tag)

    if puuid:
        print("Retrieving match IDs...")
        match_ids = get_match_ids(puuid)
        if match_ids:
            print("Saving match IDs to database...")
            df_match_id = pd.DataFrame(match_ids, columns=['Match_ID'])
            df_match_id.to_sql('MatchID_Invocador', con=engine, if_exists='replace', index=False)

            # Fetch match details asynchronously with progress bar
            print("Fetching match details...")
            match_details = asyncio.run(fetch_all_match_details(tqdm(match_ids, desc="Fetching match details")))

            # Process and save match details
            print("Processing and saving match details...")
            df_matchdetails = process_match_details(match_details)
            if not df_matchdetails.empty:
                df_matchdetails.to_sql('Detalhes_Liga_Invocador', con=engine, if_exists='replace', index=False)
                print("Match details saved successfully.")
            else:
                print("No valid match details found.")
        else:
            print("No match IDs retrieved.")
    else:
        print("Failed to retrieve PUUID.")

if __name__ == "__main__":
    main()
